In [1]:
%matplotlib inline
%load_ext memory_profiler 

In [2]:
from KnackpackData import n_items,weights,values,max_weight,max_value,encodeNumber,items_size_threshold,getDiff
from Timer import *
import matplotlib.pyplot as plt
import numpy as np
import random
from numba import njit,prange
t = Timer()

## Algorytm zachłanny GSA

Algorytm sortuje elementy w kolejności malejącej porównując wartości ${v}$ przedmiotu do jego wagi ${w}$.

$h = \frac{v}{w}$.


### Obliczenia

In [3]:
def scoreFunctionWithItemsSizes(mask, weights=weights,values=values,max_weight=max_weight,max_value=max_value):
    final_weight = 0
    score = 0 
    items = weights[mask]
    s_items = items[items <= items_size_threshold].size
    b_items = items[items > items_size_threshold].size
    for i in prange(weights.size):
        if mask[i]:
            final_weight+=weights[i]
            score+=values[i]
    return final_weight, score, s_items, b_items

def getDataWithItemsSize():
    return np.array([scoreFunctionWithItemsSizes(encodeNumber(x)) for x in prange(2**n_items)])

data_with_sizes_t = getDataWithItemsSize()
data_with_sizes = data_with_sizes_t[np.argsort(data_with_sizes_t[:, 0])] # Sort by weights
data_with_sizes = data_with_sizes[(data_with_sizes[:,0] <= max_weight)] # Delete results larger than max_weight
data_with_sizes = data_with_sizes[np.argsort(data_with_sizes[:, 1])] # Sort by score

In [4]:
# Values to weight ratios with indexes for sort
VW_ratios = values / weights
indexes = np.flip(np.argsort(VW_ratios))

In [5]:
# Sort by indexes
VW_ratios = np.take_along_axis(VW_ratios, indexes, axis=0) # Value/weight ratios
VW_weights = np.take_along_axis(weights, indexes, axis=0) # Sorted weights
VW_values = np.take_along_axis(values, indexes, axis=0) # Sorted values

In [6]:
# Calculate greedy results
greedy_score = 0
greedy_weight = 0
for i in range(len(VW_ratios)):
    if(greedy_weight + VW_weights[i]) <= max_weight:
        greedy_score += VW_values[i]
        greedy_weight += VW_weights[i]

### Porównanie wyników

W porównaniu uwzględnione zostały wyniki algorytmu zachłannego oraz 10 najlepszych wyników ze wszystkich możliwych rozwiązań.

In [7]:
print("Greedy weight: ",greedy_weight)
print("Greedy score: ",greedy_score)
print("Difference (value, weight):", getDiff(n_items,greedy_score, greedy_weight))
print("Best results (weight, value): \n",data_with_sizes[-10:,0:2])

Greedy weight:  2.7709860150069376
Greedy score:  4.871708441930054
Difference (value, weight): [0.1249741580699455, 0.12433214499306233]
Best results (weight, value): 
 [[2.65571179 4.69313976]
 [2.6979442  4.70059162]
 [2.88330288 4.74501949]
 [2.96497628 4.74671992]
 [2.95226796 4.76879224]
 [2.78004393 4.81811392]
 [2.87913456 4.8687458 ]
 [2.77098602 4.87170844]
 [2.99857711 4.92358817]
 [2.89531816 4.9966826 ]]


### Wnioski

Z powyższych danych wynika, że rozwiązanie zachłanne daje bardzo dobre wyniki, bliskie najlepszemu znanemu rozwiazaniu. Pondato metoda ta jest bardzo szybka, ponieważ wybiera jedynie lokalnie optymalne rozwiązania.

Otrzymany wynik z metody zachłannej jest 3 najlepszym wynikiem z całego zbioru danych, co pokazuje że nie jest to najlepsza metoda poszukiwania ekstremów globalnych. 


## Algorytm zachłanny GSA - alternatywa wartościowa

Algorytm sortuje elementy w kolejności malejącej po wartościach przedmiotów ${v}$ bez zwracania uwagi na jego wagi.

### Obliczenia

In [8]:
# Values indexes for sort
indexes = np.flip(np.argsort(values))

In [9]:
# Sort by indexes
S_weights = np.take_along_axis(weights, indexes, axis=0) # Sorted weights
S_values = np.take_along_axis(values, indexes, axis=0) # Sorted values

In [10]:
# Calculate greedy results
greedy_val_score = 0
greedy_val_weight = 0
for i in range(len(S_values)):
    if(greedy_val_weight + S_weights[i]) <= max_weight:
        greedy_val_score += S_values[i]
        greedy_val_weight += S_weights[i]

### Porównanie wyników

W porównaniu uwzględnione zostały wyniki algorytmu zachłannego oraz 10 najlepszych wyników ze wszystkich możliwych rozwiązań.

In [11]:
print("Greedy weight: ",greedy_val_weight)
print("Greedy score: ",greedy_val_score)
print("Difference (value, weight):", getDiff(n_items,greedy_val_score, greedy_val_weight))
print("Best results (weight, value): \n",data_with_sizes[-10:,0:2])

Greedy weight:  2.940506976253459
Greedy score:  4.459977808034045
Difference (value, weight): [0.5367047919659544, -0.04518881625345905]
Best results (weight, value): 
 [[2.65571179 4.69313976]
 [2.6979442  4.70059162]
 [2.88330288 4.74501949]
 [2.96497628 4.74671992]
 [2.95226796 4.76879224]
 [2.78004393 4.81811392]
 [2.87913456 4.8687458 ]
 [2.77098602 4.87170844]
 [2.99857711 4.92358817]
 [2.89531816 4.9966826 ]]


### Wnioski

Inne podejście do algorytmu GSA również daje wynik zadowalający, ale zdecydowanie gorszy niż klasyczny GSA.
